In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [2]:
from fake_useragent import UserAgent
base_url="https://www.ambitionbox.com/list-of-companies?page="


In [13]:
df=pd.DataFrame({})
ua = UserAgent()

headers = {
    "User-Agent": ua.random,
    "Accept-Language": "en-US,en;q=0.9",
    }
for i in range(1,500):
    url=base_url+str(i)
    response = requests.get(url, headers=headers)
    html_content = response.text
    soup=BeautifulSoup(html_content,"html.parser")
    companies=soup.find_all(class_="companyCardWrapper__companyName")
    companies=[i.text.replace("\t","").replace("\n","") for i in companies]
    # print(len(companies))
    
    # ratings
    rating=soup.find_all(class_=["rating_text--md","rating_text"])
    rating=[i.text.replace("\t","").replace("\n","") for i in rating]
    # print(len(rating))
    
    # rating count
    rating_count=soup.find_all(class_="companyCardWrapper__companyRatingCount")
    rating_count=[i.text.replace("\t","").replace("\n","").replace("(","").replace(")","") for i in rating_count]
    # print(len(rating_count))
    
    # location and sector
    loc_and_sec=soup.find_all(class_="companyCardWrapper__interLinking")
    loc_and_sec=[i.text.replace("\t","").replace("\n","").replace("(","").replace(")","").split("|") for i in loc_and_sec]
    loc,sec=[],[]
    for p in loc_and_sec:
        loc.append(p[0])
        if len(p)>1:
            sec.append(p[1])
        else:
            sec.append("")
    # reviews
    reviews=soup.find_all(class_="companyCardWrapper__ratingComparisonWrapper")
    reviews=[i.text for i in reviews]
    reviews=[review.split("Critically") for review in reviews]
    # print(reviews[8])
    if len(reviews)!=20:
        reviews+=[[""]*(20-len(reviews))]
    positive,negative=[],[]
    for p in reviews:
        # print(p)
        if len(p[0])>0:
            p[0].replace("Highly Rated For","")
        if len(p)>1 and len(p[1])>0:
            p[1].replace("Rated For","")
        positive.append(p[0])
        if len(p)>1:
            negative.append(p[1])
        else :
            negative.append("")
    # print("1",len(companies))
    # print("2",len(sec))
    # print("3",len(loc))
    # print("4",len(rating))
    # print("5",len(rating_count))
    # print("6",len(positive))
    # print("7",len(negative))
    temp_df=pd.DataFrame({
        "Company_Name":companies,
        "Sector of Company":sec,
        "Locations of company":loc,
        "Rating":rating,
        "No_of_Ratings":rating_count,
        "Positives of Company":positive,
        "Negatives of Company":negative
    })
    # print(df)
    df=pd.concat([df,temp_df],ignore_index=True)

df

,Company_Name,Sector of Company,Locations of company,Rating,No_of_Ratings,Positives of Company,Negatives of Company
0,TCS,Bangalore / Bengaluru +413 other locations,IT Services & Consulting,3.6,95.8k,"Highly Rated For Job Security, Work Life Bala...","Rated For Promotions / Appraisal, Salary & Be..."
1,Accenture,Bangalore / Bengaluru +224 other locations,IT Services & Consulting,3.8,61.5k,,"Rated For Promotions / Appraisal, Salary & Be..."
2,Wipro,Bangalore / Bengaluru +357 other locations,IT Services & Consulting,3.7,56.2k,Highly Rated For Job Security,"Rated For Promotions / Appraisal, Salary & Be..."
3,Cognizant,Hyderabad / Secunderabad +211 other locations,IT Services & Consulting,3.7,53.3k,,"Rated For Promotions / Appraisal, Salary & Be..."
4,Capgemini,Bangalore / Bengaluru +171 other locations,IT Services & Consulting,3.7,45k,"Highly Rated For Work Life Balance, Job Secur...","Rated For Promotions / Appraisal, Salary & Be..."
...,...,...,...,...,...,...,...
9975,Cades Studec Technologies,Bangalore / Bengaluru +1 other locations,IT Services & Consulting,3.9,89,"Highly Rated For Work Life Balance, Job Secur...","Rated For Promotions / Appraisal, Salary & Be..."
9976,The Akanksha Foundation,Pune +7 other locations,Education & Training,3.7,89,Highly Rated For Salary & Benefits,"Rated For Work Satisfaction, Company Culture,..."
9977,Liladhar Pasoo,Mumbai +13 other locations,Logistics,4.2,89,"Highly Rated For Company Culture, Job Securit...",
9978,Tristha Global,Chennai +5 other locations,IT Services & Consulting,4.2,89,"Highly Rated For Job Security, Salary & Benef...",


In [14]:
df=df.rename(columns={'Sector of Company': 'Locations of company', 'Locations of company': 'Sector of Company'})
df.head(10)

,Company_Name,Locations of company,Sector of Company,Rating,No_of_Ratings,Positives of Company,Negatives of Company
0,TCS,Bangalore / Bengaluru +413 other locations,IT Services & Consulting,3.6,95.8k,"Highly Rated For Job Security, Work Life Bala...","Rated For Promotions / Appraisal, Salary & Be..."
1,Accenture,Bangalore / Bengaluru +224 other locations,IT Services & Consulting,3.8,61.5k,,"Rated For Promotions / Appraisal, Salary & Be..."
2,Wipro,Bangalore / Bengaluru +357 other locations,IT Services & Consulting,3.7,56.2k,Highly Rated For Job Security,"Rated For Promotions / Appraisal, Salary & Be..."
3,Cognizant,Hyderabad / Secunderabad +211 other locations,IT Services & Consulting,3.7,53.3k,,"Rated For Promotions / Appraisal, Salary & Be..."
4,Capgemini,Bangalore / Bengaluru +171 other locations,IT Services & Consulting,3.7,45k,"Highly Rated For Work Life Balance, Job Secur...","Rated For Promotions / Appraisal, Salary & Be..."
5,HDFC Bank,Mumbai +1688 other locations,Banking,3.9,42.4k,"Highly Rated For Job Security, Skill Developm...",Rated For Promotions / Appraisal
6,Infosys,Bangalore / Bengaluru +227 other locations,IT Services & Consulting,3.6,42k,Highly Rated For Job Security,"Rated For Promotions / Appraisal, Salary & Be..."
7,ICICI Bank,Mumbai +1391 other locations,Banking,4.0,41k,"Highly Rated For Job Security, Skill Developm...",
8,HCLTech,Chennai +216 other locations,IT Services & Consulting,3.5,38.7k,Highly Rated For Job Security,"Rated For Promotions / Appraisal, Salary & Be..."
9,Tech Mahindra,Hyderabad / Secunderabad +318 other locations,IT Services & Consulting,3.5,37.3k,,"Rated For Promotions / Appraisal, Salary & Be..."


In [21]:
ans=[]
ans2=[]
for i in df["Positives of Company"]:
    i=i.replace("Highly Rated For ","")
    ans.append(i)
for i in df["Negatives of Company"]:
    i=i.replace("Rated For ","")
    ans2.append(i)
df["Positives of Company"]=ans
df["Negatives of Company"]=ans2

In [23]:
df=df[["Company_Name","Sector of Company","Rating","No_of_Ratings","Positives of Company","Negatives of Company","Locations of company"]]
df

,Company_Name,Sector of Company,Rating,No_of_Ratings,Positives of Company,Negatives of Company,Locations of company
0,TCS,IT Services & Consulting,3.6,95.8k,"Job Security, Work Life Balance","Promotions / Appraisal, Salary & Benefits, Wo...",Bangalore / Bengaluru +413 other locations
1,Accenture,IT Services & Consulting,3.8,61.5k,,"Promotions / Appraisal, Salary & Benefits",Bangalore / Bengaluru +224 other locations
2,Wipro,IT Services & Consulting,3.7,56.2k,Job Security,"Promotions / Appraisal, Salary & Benefits, Wo...",Bangalore / Bengaluru +357 other locations
3,Cognizant,IT Services & Consulting,3.7,53.3k,,"Promotions / Appraisal, Salary & Benefits, Wo...",Hyderabad / Secunderabad +211 other locations
4,Capgemini,IT Services & Consulting,3.7,45k,"Work Life Balance, Job Security","Promotions / Appraisal, Salary & Benefits, Wo...",Bangalore / Bengaluru +171 other locations
...,...,...,...,...,...,...,...
9975,Cades Studec Technologies,IT Services & Consulting,3.9,89,"Work Life Balance, Job Security","Promotions / Appraisal, Salary & Benefits",Bangalore / Bengaluru +1 other locations
9976,The Akanksha Foundation,Education & Training,3.7,89,Salary & Benefits,"Work Satisfaction, Company Culture, Work Life...",Pune +7 other locations
9977,Liladhar Pasoo,Logistics,4.2,89,"Company Culture, Job Security, Skill Developm...",,Mumbai +13 other locations
9978,Tristha Global,IT Services & Consulting,4.2,89,"Job Security, Salary & Benefits, Work Life Ba...",,Chennai +5 other locations


In [26]:
df.to_csv("companies.csv")